# Forecast

In [1]:
import re
import calendar
import pandas as pd
import numpy as np
import openpyxl as xl
from datetime import datetime
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from excel_functions import ExcelFunctions

In [2]:
def obtener_mes_actual_y_siguiente():
    # Obtener la fecha actual del sistema
    fecha_actual = datetime.now()
    
    # Obtener el número del mes actual (1 para enero, 2 para febrero, etc.)
    mes_actual = fecha_actual.month
    
    # Obtener el número del siguiente mes
    mes_siguiente = mes_actual + 1 if mes_actual < 12 else 1

    # Obtener los nombres de los meses en español
    meses_en_espanol = calendar.month_name[mes_actual].lower(), calendar.month_name[mes_siguiente].lower()

    return list(meses_en_espanol)

In [3]:
def convertir_fechas_a_meses_en_ingles(diccionario):
    diccionario_meses_en_ingles = {}

    for fecha, valores in diccionario.items():
        # Verificamos si la llave es del tipo datetime
        if isinstance(fecha, datetime):
            # Obtenemos el nombre del mes en inglés
            nombre_mes = calendar.month_name[fecha.month].lower()
            new_key = f'{nombre_mes}_{fecha.year}'
            # Agregamos la entrada al nuevo diccionario
            diccionario_meses_en_ingles[new_key] = valores
        else:
            diccionario_meses_en_ingles[fecha] = valores

    return diccionario_meses_en_ingles

In [4]:
def obtener_posicion_por_indice(diccionario, indice):
    diccionario_salida = {}

    for llave, lista in diccionario.items():
        # Verificamos si el índice es válido para la lista
        if indice < len(lista):
            diccionario_salida[llave] = lista[indice]
        else:
            # Si el índice está fuera del rango, seleccionamos el último elemento disponible
            diccionario_salida[llave] = lista[-1]

    return diccionario_salida

In [5]:
def clean_forecast_file(file, dir_location, file_name=None, return_data=True):
    '''
    Esta función recibe un archivo, ya sea ubicación para abrirlo o bytes y lo limpia.
    dir_location: el directorio donde lo va a guardar
    '''
    # Aquí están las variables necesarias
    ef = ExcelFunctions()

    # Nombres necesarios
    sheet_name = 'Resumen Pzs, Cjs, Kg-Lt'
    first_col_name = 'Item'
    wanted_cols = ['Item', 'Description']
    output_col_names = ['sku', 'descripcion']

    # Abrimos el archivo
    wb = xl.load_workbook(file, data_only=True)
    ws = wb[sheet_name]
    
    # Encontramos las dimensiones necesarias
    row_with_col_names = ef.find_value(ws, 1, first_col_name)[0]
    last_row = ef.encontrar_ultima_fila_con_datos(ws, 1, row_with_col_names)

    # Traemos los nombres de las columnas
    # Primero tenemos que ver a partir de qué columna están las cajas
    col_cajas = ef.get_columns_on_row(ws, row_with_col_names-1, num_nones_consecutivos=20)['CAJAS']
    dict_cols = ef.get_columns_on_row(ws, row_with_col_names)
    
    # Sacamos los nombres de las columnas repetidas
    # Convertimos a nombres de meses y años
    # Tomamos la segunda entrada si disponible
    otro_dict_cols = ef.get_columns_on_row_repeated_names(ws, row_with_col_names) # NEW NEW NEW
    otro_dict_cols = convertir_fechas_a_meses_en_ingles(otro_dict_cols)# new new new
    otro_dict_cols = obtener_posicion_por_indice(otro_dict_cols, 1) # new new new

    # Tenemos que agregar los nombres de los siguientes dos meses para la extraccion
    meses = obtener_mes_actual_y_siguiente()
    # Le agregamos el año actual a los meses a extraer
    otros_meses = [f'{mes}_{datetime.now().year}' for mes in meses] # NEW NEW NEW
    otro_cols_to_extract = wanted_cols + otros_meses # NEW NEW NEW
    cols_to_extract = wanted_cols + meses # Agregamos los meses deseados a extraer
    
    for i, mes in enumerate(meses):
        dict_cols[mes] = col_cajas+i

    col_names = output_col_names + meses
    extracted_data = (ef.get_data_on_columns(
         ws, 
         row_with_col_names+1, 
         otro_dict_cols, # new new new 
         last_row, 
         otro_cols_to_extract, # new new new 
         col_names
     )
    )

    # Limpiamos los datos extraídos
    for mes in meses:
        extracted_data[mes] = np.round(pd.to_numeric(extracted_data[mes]))
        
    # Agregamos el año actual a los nombres de los meses
    extracted_data = (extracted_data
     .set_index(['sku', 'descripcion'])
     .add_suffix(f' {datetime.now().year}')
     .reset_index()
    )
        
    if file_name is None:
        file_name = f'forecast_{"_".join(meses)}'
        
    # Guardamos el archivo
    data_bytes, filename = ef.save_and_download_excel_file(
        extracted_data, 
        dir_location, 
        file_name,
        sheet_name = 'Forecast', 
        n_cols_to_bold=2,
        return_data=return_data
    )
    if return_data:
        return data_bytes, filename
    
    print(f'{file_name} was updated succesfully')

In [6]:
_, _ = clean_forecast_file(
    '../raw_data/forecasts/Forecast Estadístico Colaborado Julio 2023 Nacional (1).xlsx',
    '../transformed_data',
    return_data=True
)